![](https://i.imgur.com/wcCJf9v.png)

# Imports 📚

Let's import the libraries we will be using in this notebook

In [ ]:
# Data Import on Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing processing libraries
import numpy as np
import pandas as pd

# Importing Visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import palettable.scientific.sequential as palette
from palettable.tableau import Tableau_20
import matplotlib.gridspec as gridspec

import warnings
warnings.filterwarnings("ignore")

sns.set_style('white')

In [ ]:
df = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv')
df = df.iloc[1:] # Removing the questions from the data

df20 = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')
df20 = df20.iloc[1:] # Removing the questions from the data

df19a = pd.read_csv('../input/kaggle-survey-2019/multiple_choice_responses.csv')
df19a = df19a.iloc[1:] # Removing the questions from the data

df19b = pd.read_csv('../input/kaggle-survey-2019/other_text_responses.csv')
df19b = df19b.iloc[1:] # Removing the questions from the data

# My Plan

The Kaggle Machine Learning and Data Science Survey has collected huge amounts of information on users across the world and there are several burning questions we might want answered. One such question I had was the differences in Kaggle users across countries. Now this is a broad questions in itself and has various caveats we need to look at.

How do user behaviours change when we look at a developing country (like India) as opposed to a developed country (like USA)? What are the differences not only in the gender split but their ages, education, earnings, etc. 

As we take a deeper dive into the comparisons of these countries, we also need to make sure our results are reproducible. Are the similarities/differences we see unique to India and USA or are they seen in other comparisons as well (say, Nigeria vs Japan, Brazil vs UK).

Furthermore, while we're talking about similarities between one group of countries, do we see a trend here? Do countries with a similar population have unique trends? What about ones with similar GDP?

While looking at these macro comparisons across the world, there was a small-question I wanted to answer. Of course, different countries shouldn't have different uses of models (not significant at least). However, different industries would use require custom models for their own data and I take a quick look at the models each industry prefers. Also, I look at how usage of models differs across company sizes.

Finally, we conclude with our results from the comparisons of the countries.

In [ ]:
df.head(5)

## World Plots

Let's look at some trends about Kaggle users across the world: 
- Gender distributions
- Distribution of users by their Education Level
- Distribution of users by their Job Title

In [ ]:
def gender_split():
    male_df = df[df['Q2'] == 'Man']
    female_df = df[df['Q2'] == 'Woman']
    other_df = df[(df['Q2'] != 'Man') & (df['Q2'] != 'Woman')]
    return male_df, female_df, other_df
male_df, female_df, other_df = gender_split()

fig = plt.figure(figsize=(17, 8))
spec = gridspec.GridSpec(ncols=3, nrows=2, figure=fig)

# Gender Split
ax = fig.add_subplot(spec[0,0])
gender_idx = ['Male', 'Woman', 'Other']
gender_vals = df['Q2'].value_counts().values
gender_vals[2] = gender_vals[2:].sum()

circle = plt.Circle( (0,0), 0.7, color='white')
ax.pie(gender_vals[:3], explode = (0, 0.1, 0.2), labels = gender_idx, colors=palette.Acton_4.hex_colors)
p=plt.gcf()
p.gca().add_artist(circle)
for s in ['top', 'right', 'bottom', 'left']:
    ax.spines[s].set_visible(False)

# Education Split
edu_ser= df['Q4'].value_counts()
edu_ser = edu_ser.reindex(['No formal education past high school','Professional doctorate',
                           'Bachelor’s degree',"Master’s degree",'Doctoral degree',
                           'Some college/university study without earning a bachelor’s degree','I prefer not to answer'])
edu_idx = edu_ser.index
edu_vals = edu_ser.values
edu_idx = ['High School','Doctorate',
                       'Bachelor’s',"Master’s",'Doctoral degree',
                       'Other','No Answer']
ax1 = fig.add_subplot(spec[0,2])
ax1.bar( edu_idx, edu_vals, color=palette.Acton_7.hex_colors)
ax1.set_xticklabels(edu_idx, rotation=40)
ax1.set_yticklabels([])

for s in ['top', 'right', 'bottom', 'left']:
    ax1.spines[s].set_visible(False)

# Current Role v Gender    
ax1 = fig.add_subplot(spec[1,:])
role_ser= df['Q5'].value_counts()
role_ser = role_ser.reindex(['Developer Relations/Advocacy','Statistician',
                             'Data Engineer','Business Analyst','Research Scientist','Data Analyst','Software Engineer','Student',
              'Data Scientist','Other','Currently not employed','Machine Learning Engineer','Program/Project Manager','Product Manager','DBA/Database Engineer'])
role_idx = role_ser.index
role_vals = role_ser.values
role_idx = ['Developer Relations','Statistician',
                             'Data Engineer','Business Analyst','Research Scientist','Data Analyst','Software Engineer','Student',
              'Data Scientist','Other','Unemployed','ML Engineer','Project Manager','Product Manager','Database Engineer']

ax1.bar( role_idx, role_vals, color=palette.Acton_15.hex_colors)
ax1.set_xticklabels(role_idx, rotation=40)
ax1.set_yticklabels([])

for s in ['top', 'right', 'bottom', 'left']:
    ax1.spines[s].set_visible(False)


# Text

fig.text(0.28, 0.75, 'Distributions of the respondants', fontsize=17, fontweight='bold', fontfamily='sans-serif')
fig.text(0.28, 0.52, 
'''From the figures below, clearly, most of the repondants are originated
from India and more significantly, the majority are male. We also take a 
quick look at their education backgrounds and their experience. Don't 
worry, we will dive into the correlations between these features soon. 
As mentioned earlier, our analysis here is geared towards comparisons in 
developed vs developing countries so we're looking at a few variables that 
might be important later.
'''
, fontsize=14, fontweight='light', fontfamily='sans-serif')

fig.tight_layout() 
plt.subplots_adjust(wspace=1)
plt.show()






# Developed vs Developing Coutries

Now that we've looked at some distributions of the Kaggle users across the world, let's get into the question we are trying to analyse today - How do these users differ across countries, especially differences between developed and developing economies.


In [ ]:
display(df.groupby(['Q3']).count()['Q1'].sort_values(ascending=False)[:10])

### Why

Our major concern we have from our world plots earlier is that since a majority of the data comes from India, our plots could be misleading. So, we will compare the data from a developing country (India) against a developed country (USA) and see how the results hold up. 

Are Kaggle users still majorly "male undergraduate students"?

In [ ]:
def create_country_plots(df, row_num, color1, color2, color3, name, fig, spec):

    # Gender Split
    # ------------------------
    ax = fig.add_subplot(spec[row_num,0])
    gender_idx = ['Male', 'Woman', 'Other']
    gender_vals = df['Q2'].value_counts().values
    gender_vals[2] = gender_vals[2:].sum()

    circle = plt.Circle( (0,0), 0.7, color='white')
    ax.pie(gender_vals[:3], explode = (0, 0.1, 0.2), labels = gender_idx, colors=color1)
    p=plt.gcf()
    p.gca().add_artist(circle)
    for s in ['top', 'right', 'bottom', 'left']:
        ax.spines[s].set_visible(False)

    # Education Split
    # ------------------------
    edu_ser= df['Q4'].value_counts()
    edu_ser = edu_ser.reindex(['No formal education past high school','Professional doctorate',
                               'Bachelor’s degree',"Master’s degree",'Doctoral degree',
                               'Some college/university study without earning a bachelor’s degree','I prefer not to answer'])
    edu_idx = edu_ser.index
    edu_vals = edu_ser.values
    edu_idx = ['High School','Doctorate',
                           'Bachelor’s',"Master’s",'Doctoral degree',
                           'Other','No Answer']
    ax1 = fig.add_subplot(spec[row_num,1])
    ax1.bar( edu_idx, edu_vals, color=color2)
    ax1.set_xticklabels(edu_idx, rotation=70)
    ax1.set_yticklabels([])

    for s in ['top', 'right', 'bottom', 'left']:
        ax1.spines[s].set_visible(False)
        
    # Role Split    
    # ------------------------
    ax1 = fig.add_subplot(spec[row_num,2])
    role_ser_us= df['Q5'].value_counts()
    role_ser_us = role_ser_us.reindex(['Developer Relations/Advocacy','Statistician',
                                 'Data Engineer','Business Analyst','Research Scientist','Data Analyst','Software Engineer','Student',
                  'Data Scientist','Other','Currently not employed','Machine Learning Engineer','Program/Project Manager','Product Manager','DBA/Database Engineer'])
    role_idx_us = role_ser_us.index
    role_vals_us = role_ser_us.values
    role_idx_us = ['Dev Relations','Statistician',
                                 'Data Eng','Business Analyst','Research Scientist','Data Analyst','SWE','Student',
                  'Data Scientist','Other','Unemployed','ML Engineer','Project Manager','Product Manager','Database Engineer']

    ax1.bar( role_idx_us, role_vals_us, color=color3)
    ax1.set_xticklabels(role_idx_us, rotation=70)
    ax1.set_yticklabels([])

    for s in ['top', 'right', 'bottom', 'left']:
        ax1.spines[s].set_visible(False)

    # Experience Split    
    # ------------------------
    ax1 = fig.add_subplot(spec[row_num,3])
    exp_ser = df['Q6'].value_counts()
    exp_ser = exp_ser.reindex(['< 1 years', '1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years'])
    exp_index = exp_ser.index
    exp_vals = exp_ser.values
    ax1.bar( exp_index, exp_vals, color=color2)
    ax1.set_xticklabels(exp_index, rotation=70)
    ax1.set_yticklabels([])
    if name == 'Nigeria':
        ax1.set_xlabel("Coding Experience")
    for s in ['top', 'right', 'bottom', 'left']:
        ax1.spines[s].set_visible(False)

In [ ]:
ind_df = df[df['Q3'] == 'India']
us_df = df[df['Q3'] == 'United States of America']

fig = plt.figure(figsize=(17, 12))
spec = gridspec.GridSpec(ncols=3, nrows=4, figure=fig)

# Gender Split
ax = fig.add_subplot(spec[0,0])
gender_idx = ['Male', 'Woman', 'Other']
gender_vals = ind_df['Q2'].value_counts().values
gender_vals[2] = gender_vals[2:].sum()
circle = plt.Circle( (0,0), 0.7, color='white')
ax.pie(gender_vals[:3], explode = (0, 0.1, 0.2), labels = gender_idx, colors=palette.Acton_4.hex_colors)
p=plt.gcf()
p.gca().add_artist(circle)

# ------------------------

ax2 = fig.add_subplot(spec[1,0])
gender_vals_us = us_df['Q2'].value_counts().values
gender_vals_us[2] = gender_vals_us[2:].sum()
circle = plt.Circle( (0,0), 0.7, color='white')
ax2.pie(gender_vals_us[:3], explode = (0, 0.1, 0.2), labels = gender_idx, colors=palette.Batlow_4.hex_colors)
p=plt.gcf()
p.gca().add_artist(circle)

# ------------------------
# ------------------------

# Education Split
edu_ser= ind_df['Q4'].value_counts()
edu_ser = edu_ser.reindex(['No formal education past high school','Professional doctorate',
                           'Bachelor’s degree',"Master’s degree",'Doctoral degree',
                           'Some college/university study without earning a bachelor’s degree','I prefer not to answer'])
edu_idx = edu_ser.index
edu_vals = edu_ser.values
edu_idx = ['High School','Doctorate',
                       'Bachelor’s',"Master’s",'Doctoral degree',
                       'Other','No Answer']
ax1 = fig.add_subplot(spec[0,2])
ax1.bar( edu_idx, edu_vals, color=palette.Acton_7.hex_colors)
ax1.set_xticklabels(edu_idx, rotation=40)
ax1.set_yticklabels([])

for s in ['top', 'right', 'bottom', 'left']:
    ax1.spines[s].set_visible(False)


# ------------------------

edu_ser_us= us_df['Q4'].value_counts()
edu_ser_us = edu_ser_us.reindex(['No formal education past high school','Professional doctorate',
                           'Bachelor’s degree',"Master’s degree",'Doctoral degree',
                           'Some college/university study without earning a bachelor’s degree','I prefer not to answer'])
edu_idx_us = edu_ser_us.index
edu_vals_us = edu_ser_us.values
edu_idx_us = ['High School','Doctorate',
                       'Bachelor’s',"Master’s",'Doctoral degree',
                       'Other','No Answer']
ax1 = fig.add_subplot(spec[1,2])
ax1.bar( edu_idx_us, edu_vals_us, color=palette.Batlow_7.hex_colors)
ax1.set_xticklabels(edu_idx_us, rotation=40)
ax1.set_yticklabels([])

for s in ['top', 'right', 'bottom', 'left']:
    ax1.spines[s].set_visible(False)

# ------------------------
# ------------------------

# Current Role     
ax1 = fig.add_subplot(spec[2,:])
role_ser= ind_df['Q5'].value_counts()
role_ser = role_ser.reindex(['Developer Relations/Advocacy','Statistician',
                             'Data Engineer','Business Analyst','Research Scientist','Data Analyst','Software Engineer','Student',
              'Data Scientist','Other','Currently not employed','Machine Learning Engineer','Program/Project Manager','Product Manager','DBA/Database Engineer'])
role_idx = role_ser.index
role_vals = role_ser.values
role_idx = ['Developer Relations','Statistician',
                             'Data Engineer','Business Analyst','Research Scientist','Data Analyst','Software Engineer','Student',
              'Data Scientist','Other','Unemployed','ML Engineer','Project Manager','Product Manager','Database Engineer']

ax1.bar( role_idx, role_vals, color=palette.Acton_15.hex_colors)
ax1.set_xticklabels(role_idx, rotation=40)
ax1.set_yticklabels([])
for s in ['top', 'right', 'bottom', 'left']:
    ax1.spines[s].set_visible(False)
    
    
# ------------------------
ax1 = fig.add_subplot(spec[3,:])
role_ser_us= us_df['Q5'].value_counts()
role_ser_us = role_ser_us.reindex(['Developer Relations/Advocacy','Statistician',
                             'Data Engineer','Business Analyst','Research Scientist','Data Analyst','Software Engineer','Student',
              'Data Scientist','Other','Currently not employed','Machine Learning Engineer','Program/Project Manager','Product Manager','DBA/Database Engineer'])
role_idx_us = role_ser_us.index
role_vals_us = role_ser_us.values
role_idx_us = ['Developer Relations','Statistician',
                             'Data Engineer','Business Analyst','Research Scientist','Data Analyst','Software Engineer','Student',
              'Data Scientist','Other','Unemployed','ML Engineer','Project Manager','Product Manager','Database Engineer']

ax1.bar( role_idx_us, role_vals_us, color=palette.Batlow_15.hex_colors)
ax1.set_xticklabels(role_idx_us, rotation=40)
ax1.set_yticklabels([])

for s in ['top', 'right', 'bottom', 'left']:
    ax1.spines[s].set_visible(False)


# Text
fig.text(0.15, 0.43, 'INDIA', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#5D3F6A')
fig.text(0.12, 0.17, 'UNITED STATES', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#407A52')
fig.text(0.28, 0.89, 'Respondant Comparison - India vs USA', fontsize=17, fontweight='bold', fontfamily='sans-serif')
fig.text(0.28, 0.58, 
'''Well, this was a little unexpected. While I did predict differences in 
the distributions, I expected a much bigger difference in the gender gap 
between the two countries.
However, we see that while the gender split remains almost the same (with a
slight increase in women in USA), Kaggle users from the United States seem 
to be more educated (or, specialised). They tend to have a Master's degree 
or a Doctoral degree as compared to the primarily undergraduate students
from India.
This also falls in line with our 
visualisations in the third plot. USA has more users in Kaggle that are 
working professionally.

However, will these results hold up in other comparisons? Are these
differences common to all developing vs developed countries?
'''
, fontsize=14, fontweight='light', fontfamily='sans-serif')

fig.tight_layout() 
plt.subplots_adjust(wspace=1)
plt.show()

# Follow-Up Questions

Now that we've seen the differences between India and USA, there are two follow-up questions that arise:
- Are these results are consistent across other developing vs developed countries?
- What are the differences/similarities between similar groups (say, India and China; US and Canada)?

For the first question, I will take a look at China and Brazil (developing) alongside Japan and the UK (developed) to see what kind of insights we can extract.
Then, using the same 4 countries alongside India and US from before, we'll see if there exists differences within each group as well.

In [ ]:
nig_df = df[df['Q3'] == 'Nigeria']
brazil_df = df[df['Q3'] == 'Brazil']
uk_df = df[df['Q3'] == 'United Kingdom of Great Britain and Northern Ireland']
japan_df = df[df['Q3'] == 'Japan']

fig = plt.figure(figsize=(18, 20))
spec = gridspec.GridSpec(ncols=4, nrows=5, figure=fig)


create_country_plots(nig_df, 0, palette.Acton_3.hex_colors, palette.Acton_7.hex_colors,palette.Acton_15.hex_colors, "Nigeria", fig, spec)
create_country_plots(brazil_df, 1, palette.Bamako_3.hex_colors, palette.Bamako_7.hex_colors,palette.Bamako_15.hex_colors, "Brazil", fig, spec)
create_country_plots(uk_df, 3, palette.Batlow_3.hex_colors, palette.Batlow_7.hex_colors,palette.Batlow_15.hex_colors, "United Kingdom", fig, spec)
create_country_plots(japan_df, 4, palette.Davos_3.hex_colors, palette.Davos_7.hex_colors,palette.Davos_15.hex_colors, "Japan", fig, spec)


            
# # Age Split    
# # ------------------------ 
# ax1 = fig.add_subplot(spec[2,0])

# nig_age_ser = nig_df['Q1'].value_counts()
# nig_age_ser = nig_age_ser.reindex(['18-21','22-24',  '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-69'])
# nig_age_vals = nig_age_ser.values

# brazil_age_ser = brazil_df['Q1'].value_counts()
# brazil_age_ser = brazil_age_ser.reindex(['18-21','22-24',  '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-69'])
# brazil_age_index = brazil_age_ser.index
# brazil_age_vals = brazil_age_ser.values
# X_axis = np.arange(len(brazil_age_ser))

# ax1.bar( X_axis-0.2, nig_age_vals, 0.4, color=palette.Acton_10.hex_colors)
# ax1.bar( X_axis+0.2, brazil_age_vals,0.4, color=palette.Bamako_10.hex_colors)
# ax1.set_xticklabels(X_axis, rotation=70)
# plt.xticks(X_axis, brazil_age_index)
# ax1.set_yticklabels([])
# ax2.set_title("Age Groups")
# for s in ['top', 'right', 'bottom', 'left']:
#     ax1.spines[s].set_visible(False)

    

# ax2 = fig.add_subplot(spec[2,3])

# uk_age_ser = uk_df['Q1'].value_counts()
# uk_age_ser = uk_age_ser.reindex(['18-21','22-24',  '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-69'])
# uk_age_vals = uk_age_ser.values

# japan_age_ser = japan_df['Q1'].value_counts()
# japan_age_ser = japan_age_ser.reindex(['18-21','22-24',  '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-69'])
# japan_age_vals = japan_age_ser.values

# ax2.bar( X_axis-0.2, uk_age_vals, 0.4, color=palette.Batlow_10.hex_colors)
# ax2.bar( X_axis+0.2, japan_age_vals,0.4, color=palette.Davos_10.hex_colors)
# ax2.set_xticklabels(X_axis, rotation=70)
# plt.xticks(X_axis, brazil_age_index)
# ax2.set_yticklabels([])
# ax2.set_xlabel("Age Groups")
# for s in ['top', 'right', 'bottom', 'left']:
#     ax2.spines[s].set_visible(False)
# fig.text(0.10, 0.58, 'Nigeria vs Brazil', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#5D3F6A')
# fig.text(0.87, 0.58, 'UK vs Japan', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#F58F84')


fig.text(0.46, 0.93, 'NIGERIA', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#5D3F6A')
fig.text(0.47, 0.73, 'BRAZIL', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#407A52')
fig.text(0.44, 0.33, 'UNITED KINGDOM', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#F58F84')
fig.text(0.48, 0.15, 'JAPAN', fontsize=17, fontweight='bold', fontfamily='sans-serif', color='#1F4788')

fig.text(0.28, 0.57, 'Kaggle Users in Developing vs Developed Economies', fontsize=17, fontweight='bold', fontfamily='sans-serif')
fig.text(0.18, 0.42, 
'''Right off the bat, there's a few points we notice. Some of our conclusions drawn from our US-India comparisons still hold true 
- Developing economies like Nigeria and Brazil have a higher number of undergraduate students while Japan and the UK have 
primarily graduate students.

However, while our conclusions hold true for some, we do notice differences in some categories. For example, Nigeria majorly has 
users with 0-3 years of experience and almost none with over 10 years while the other countries have a more even spread. 

Alternatively, when we look at professions, Brazil follows a distribution similar to UK while Nigeria and Japan seem to 
have similar proportions. We'll be exploring these in depth in the next section.

What really caught my eye here, however, is the Gender distributions in Japan. While the gender split in other countries seem 
to hover at arounf 75-80% males, Japan has an overwhelming male population of almost 90%
'''
, fontsize=14, fontweight='light', fontfamily='sans-serif')


fig.tight_layout() 
plt.subplots_adjust(wspace=0.3)
plt.show()

# Initial Conclusions from Spread across Countries

As noted in the plot, we saw that some of conclusions are consistent with the US-India model. However, we do see some inconsistencies which we will be analysing soon. 

The code block for the plots above includes a section that analyses Age groups of the 4 countries as well but I've commented those out since they weren't as beneficial as the other plots.

# ML Models in Industries

Alright, you're probably bored of seeing the same visualisations for different countries. Let's take a look at the models people prefer. 

How do usage of ML models differ in industries and company sizes? While we aren't performing hypothesis testing in this notebook, my intial thoughts are that smaller companies that don't usually have a dedicated data team probably don't work with these models much.

Also, for the Industry, one would expect organisations in Computers/Technology to use Neural Network architechture with the Financial industry using basic Regression and Classification models.

In [ ]:
# Code from Ruchi798's notebook

def visualize_relation(start_slice, end_slice, new_col_names, old_col, new_col, xlabel, title, p1,p2):
    df_sliced = df.iloc[:,start_slice:end_slice].iloc[1:]

    df_sliced = df_sliced.rename(columns=new_col_names).fillna(0).replace('[^\\d]',1, regex=True)
    df_sliced = df_sliced.join(df[old_col])

    df_sliced_stats = pd.DataFrame()
    for col in df_sliced.columns[:-1]:
        df_sliced_stats[col] = df_sliced.groupby(old_col)[col].mean().values

    df_sliced = df_sliced.rename(columns={old_col:new_col})
    df_sliced_stats.index = df_sliced.groupby(new_col)[list(new_col_names.items())[0][1]].mean().index

    cmap = sns.diverging_palette(p1, p2, as_cmap=True)
    display(df_sliced_stats.style.background_gradient(cmap, axis=0).format("{:.0%}"))

    df_sliced_stats[new_col] = df_sliced_stats.index
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    for i in range(len(df_sliced_stats.columns[:-1])):
        color = Tableau_20.hex_colors[i]
        col = df_sliced_stats.columns[i]
        df_sliced_stats.plot(kind="scatter", x=col,y=new_col, color=color, label=col,ax=ax, s=100)
    ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
    ax.set_xlabel(xlabel)
    ax.legend(loc='upper right',bbox_to_anchor=(1.35, 1), frameon=False)
    ax.set_title(title,font="Serif")
    plt.show()

In [ ]:
new_col_names ={'Q17_Part_1': 'Lin/Log Reg',
                'Q17_Part_2': 'Dec Trees/Random For',
                'Q17_Part_3': 'GBM',
                'Q17_Part_4': 'Bayesian Approaches',
                'Q17_Part_5': 'Evolutionary Approaches',
                'Q17_Part_6': 'Dense NN',
                'Q17_Part_7': 'CNN',
                'Q17_Part_8': 'GAN',
                'Q17_Part_9': 'RNN',
                'Q17_Part_10': 'Transformer Networks',
                'Q17_Part_11': 'None',
                'Q17_OTHER': 'Other'
                }

visualize_relation(90,102, new_col_names, 'Q20', 'Industry', "Usage of Models", "Industry vs Model Used", 240, 10)
visualize_relation(90,102, new_col_names, 'Q21', 'Company Size', "Usage of Models", "Company Size vs Model Used", 240, 10)

The correlations seem to be in-line with our initial thoughts. Additionally, Security and Defense Industries focus on GANs and CNNs probably owing to their heavy involvement with facial recognition softwares.

The distributions of model usage seem consistent across the board for different industries as well as various company sizes.

# Next Steps

This has just been an initial dive into usage between different countries. There are still some insights we haven't looked at and I will be adding them to this notebook soon such as:

- We saw similarities between groups of developing and developed countries respectively. But, what makes them stand out. In other words, how does an economy like China with its large population hold up against Nigeria. Similarly, how does USA compare to UK, Canada and other first world nations?
- How do salaries differ across countries and their correlations with different Job Titles
- Are there certain industries that do better in some countries and are relatively higher paying?
- Time Series -- How have all of these trends changed over the years (Compairsons with Kaggle survey results from 2019 and 2020)?

In [ ]:
# Next Steps:

# Comparisons of Industries and Salaries around the World
# Differences in Countries within each Group
# Changes in the data from 2019 - 2021

# Concluding Remarks

A huge thank you to all the other amazing Kaggle contributors whose notebooks and stories inspired me to make this. I have referred to code from a couple other contributors and I'm grateful for their input!

If reading this notebook, helped spur an idea in you and there's something you'd like me to analyse, leave a comment and I'll be sure to include it here! Thanks for Reading!